Batch detection using SAHI tools for tiled predictions but full image outputs; yield COCO format annotations
A little hacked together? Yes, but it works!
Reference: https://colab.research.google.com/github/obss/sahi/blob/main/demo/inference_for_yolov5.ipynb?v=1#scrollTo=yXyP3T0wyGtF

In [ ]:
import os
from sahi.utils.yolov5 import download_yolov5s6_model
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image
import pandas as pd

def process_images_in_directory(image_directory, save_directory):
    os.makedirs(save_directory, exist_ok=True)
    yolov5_model_path = 'models\\best.pt' #specify full model path
    download_yolov5s6_model(destination_path=yolov5_model_path)

    detection_model = AutoDetectionModel.from_pretrained(
        model_type='yolov5', #can accommodate other architectures
        model_path=yolov5_model_path,
        confidence_threshold=0.3, #specify confidence threshold as desired
        device="cuda:0" #to use GPU
    )

    all_anns = []

    for img_filename in os.listdir(image_directory):
        if img_filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img_path = os.path.join(image_directory, img_filename)

            result = get_sliced_prediction(
                img_path,
                detection_model,
                slice_height=456, #Can adjust tile size 
                slice_width=781,
                overlap_height_ratio=0.1, #Can adjust tile overlap
                overlap_width_ratio=0.1
            )

            coco = result.to_coco_annotations()
            for annotation in coco:
                annotation['image_id'] = img_filename

            all_anns.extend(coco)

        df = pd.DataFrame(all_anns)
        savepath = os.path.join(save_directory, "coco_detections.csv") #change name if desired
        df.to_csv(savepath, index=False)

image_directory = ""  # Replace with the path to the directory containing images
save_directory = ""  # Replace with the desired output directory for the COCO annotations
process_images_in_directory(image_directory, save_directory)